# Results of simulation of different fairness policies

These experiments use accumulated deficits to try to ensure that applications always receive their computed allocation of GPU time, even in the event of new jobs coming in and old jobs finishing, by keeping track of the difference between the GPU time the application should have received, and the GPU time the application actually received. Allocation of jobs to GPUs is performed in a round-based fashion, with GPUs instructed to run jobs for a fixed interval of time on all GPUs.

# Import statements

In [ ]:
# Imports for plotting.
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.ticker import MultipleLocator
from pylab import *
import seaborn as sns
from matplotlib import rc
rc('text',
   usetex=True)
sns.set_style('ticks')
font = {
    'font.family':'Times New Roman',
    'font.weight': 200,
    'font.size': 10,
}
sns.set_style(font)
flatui = ['#002A5E', '#FD151B', '#8EBA42', '#348ABD', '#988ED5', '#BDB76B', '#8EBA42', '#FFB5B8']
sns.set_palette(flatui)
paper_rc = {
    'lines.linewidth': 3,
    'lines.markersize': 10,
    'mathtext.fontset': 'custom',
    'mathtext.rm': 'Times New Roman',
    'mathtext.bf': 'Times New Roman:bold',
}
sns.set_context("paper", font_scale=2,  rc=paper_rc)
current_palette = sns.color_palette()

In [ ]:
# Other imports.
import os
import pickle
import re

In [ ]:
LOAD_FROM_PICKLED_DATA = False

In [ ]:
reading_f = None
writing_f = None
if LOAD_FROM_PICKLED_DATA:
    reading_f = open("throughput_estimation/data.pickle", 'rb')
else:
    writing_f = open("throughput_estimation/data.pickle", 'wb')

# Get logfile paths

In [ ]:
def get_logfile_paths_helper(directory_name):
    logfile_paths = []
    for root, _, file_names in os.walk(directory_name):
        if len(file_names) > 0:
            logfile_paths.extend(
                [os.path.join(root, file_name)
                 for file_name in file_names])
    return logfile_paths

def get_logfile_paths(directory_name):
    logfile_paths = []
    for logfile_path in get_logfile_paths_helper(directory_name):
        m = re.match(
            r'.*v100=(\d+)\.p100=(\d+)\.k80=(\d+)/(.*)/'
             'profiling_percentage=(\d+\.\d+)/'
             'num_reference_models=(\d+)/'
             'seed=(\d+)/lambda=(\d+\.\d+)\.log', logfile_path)
        if m is None: continue
        v100s = int(m.group(1))
        p100s = int(m.group(2))
        k80s = int(m.group(3))
        policy = m.group(4)
        profiling_percentage = float(m.group(5))
        num_reference_models = int(m.group(6))
        seed = int(m.group(7))
        l = float(m.group(8))
        logfile_paths.append((v100s, p100s, k80s, policy,
                              profiling_percentage,
                              num_reference_models,
                              seed,
                              l, logfile_path))
    return logfile_paths

In [ ]:
all_completion_algos = ["PMF"]
all_seeds = set([0, 1, 2])
all_policies = ['max_min_fairness_perf', 'max_min_fairness_packed']
logfile_paths = sorted(get_logfile_paths(
    "/lfs/1/keshav2/gpusched/scheduler/logs/throughput_estimation"))

# Plotting functions

In [ ]:
def prune(logfile_paths, v100s, p100s, k80s, policy,
          profiling_percentage, num_reference_models,
          seed=None):
    if seed is None:
        return sorted([(x[7], x[8], x[6]) for x in logfile_paths
                       if x[0] == v100s and x[1] == p100s and
                       x[2] == k80s and x[3] == policy and
                       x[4] == profiling_percentage and
                       x[5] == num_reference_models])
    else:
        return sorted([(x[7], x[8]) for x in logfile_paths
                       if x[0] == v100s and x[1] == p100s and
                       x[2] == k80s and x[3] == policy and
                       x[7] == seed])

In [ ]:
def plot_input_job_rate_vs_average_jct(average_jcts, output_filename=None):
    data = {"input_job_rate": [], 'average_jct': [], "seed": [],
            "label": []}
    
    policies = sorted(average_jcts.keys())
    for i, policy in enumerate(policies):
        for lam in average_jcts[policy]:
            input_job_rate = 3600.0 / lam
            for seed in average_jcts[policy][lam]:
                average_jct = average_jcts[policy][lam][seed]
                if average_jct:
                    data["input_job_rate"].append(input_job_rate)
                    data["average_jct"].append(average_jct)
                    data["seed"].append(seed)
                    if 'packed' in policy:
                        label = 'LAS+perf+packed'
                    else:
                        label = 'LAS+perf'
                    if 'estimated' in policy:
                        label += ' [ET]'
                    data["label"].append(label)
    plt.figure(figsize=(8, 3.5))
    ax = plt.subplot2grid((1, 1), (0, 0), colspan=1)
    sns.lineplot(x='input_job_rate', y='average_jct', style='label',
                 hue='label',
                 data=data, ci='sd',
                 markers=True)
    ax.set_xlabel("Input job rate (jobs/hr)")
    ax.set_ylabel("Average JCT (hours)")
    plt.ylim(0, 300)
    sns.despine()
    plt.legend()  
    if output_filename is not None:
        with PdfPages(output_filename) as pdf:
            pdf.savefig(bbox_inches='tight')
    plt.show()

# Plot average JCT versus input job rate

In [ ]:
def get_average_jct(logfile_path, min_job_id=None,
                   max_job_id=None):
    job_completion_times = []
    with open(logfile_path, 'rb') as f:
        f.seek(-8192, os.SEEK_END)
        text = f.read().decode('utf-8')
        lines = text.split('\n')
        for line in lines[-5:]:
            m = re.match(r'Average job completion time: (\d+\.\d+) seconds', line)
            if m is not None:
                return float(m.group(1)) / 3600
    return None

In [ ]:
def compute_metric(v100s, p100s, k80s, policies,
                   profiling_percentage, num_reference_models,
                   seeds):
    data = {}
    policies_ = []
    if reading_f is not None:
        data = pickle.load(reading_f)
    else:
        for policy in policies:
            policies_.append(policy)
            if 'packed' in policy:
                policies_.append(policy + '_estimated')
        for i, policy in enumerate(policies_):
            if '_estimated' in policy:
                profiling_percentage_ = profiling_percentage
                policy_ = policy.split('_estimated')[0]
            else:
                profiling_percentage_ = 0.0
                policy_ = policy
            data[policy] = {}
            relevant_logfile_paths = list(reversed(prune(
                logfile_paths, v100s, p100s, k80s, policy_,
                profiling_percentage_, num_reference_models)))
            for logfile_path in relevant_logfile_paths:
                lam, path, seed = logfile_path
                if seed not in all_seeds:
                    continue
                if lam not in data[policy]:
                    data[policy][lam] = {}
                average_jct = get_average_jct(path, min_job_id=4000, max_job_id=5000)
                if average_jct is not None:
                    data[policy][lam][seed] = average_jct
        pickle.dump(data, writing_f)
    return data

In [ ]:
profiling_percentage = 1.0/64
print("V100s=64, P100s=0, K80s=0; Jobs 4000-5000")
print("profiling_percentage=%f" % profiling_percentage)
average_jcts = compute_metric(v100s=64, p100s=0, k80s=0,
                              policies=all_policies,
                              profiling_percentage=profiling_percentage,
                              num_reference_models=16,
                              seeds=all_seeds)
plot_input_job_rate_vs_average_jct(average_jcts,
                                   output_filename='throughput_estimation/profiling_percentage=0.015625_v100=64_p100=0_k80=0.pdf')

In [ ]:
profiling_percentage = 2.0/64
print("V100s=64, P100s=0, K80s=0; Jobs 4000-5000")
print("profiling_percentage=%f" % profiling_percentage)
average_jcts = compute_metric(v100s=64, p100s=0, k80s=0,
                              policies=all_policies,
                              profiling_percentage=profiling_percentage,
                              num_reference_models=16,
                              seeds=all_seeds)
plot_input_job_rate_vs_average_jct(average_jcts,
                                   output_filename='throughput_estimation/profiling_percentage=0.03125_v100=64_p100=0_k80=0.pdf')

In [ ]:
profiling_percentage = 4.0/64
print("V100s=64, P100s=0, K80s=0; Jobs 4000-5000")
print("profiling_percentage=%f" % profiling_percentage)
average_jcts = compute_metric(v100s=64, p100s=0, k80s=0,
                              policies=all_policies,
                              profiling_percentage=profiling_percentage,
                              num_reference_models=16,
                              seeds=all_seeds)
plot_input_job_rate_vs_average_jct(average_jcts,
                                   output_filename='throughput_estimation/profiling_percentage=0.0625_v100=64_p100=0_k80=0.pdf')

In [ ]:
profiling_percentage = 1.0/64
print("V100s=36, P100s=36, K80s=36; Jobs 4000-5000")
print("profiling_percentage=%f" % profiling_percentage)
average_jcts = compute_metric(v100s=36, p100s=36, k80s=36,
                              policies=all_policies,
                              profiling_percentage=profiling_percentage,
                              num_reference_models=16,
                              seeds=all_seeds)
plot_input_job_rate_vs_average_jct(average_jcts,
                                   output_filename='throughput_estimation/profiling_percentage=0.015625_v100=36_p100=36_k80=36.pdf')

In [ ]:
profiling_percentage = 2.0/64
print("V100s=36, P100s=36, K80s=36; Jobs 4000-5000")
print("profiling_percentage=%f" % profiling_percentage)
average_jcts = compute_metric(v100s=36, p100s=36, k80s=36,
                              policies=all_policies,
                              profiling_percentage=profiling_percentage,
                              num_reference_models=16,
                              seeds=all_seeds)
plot_input_job_rate_vs_average_jct(average_jcts,
                                   output_filename='throughput_estimation/profiling_percentage=0.03125_v100=36_p100=36_k80=36.pdf')

In [ ]:
profiling_percentage = 4.0/64
print("V100s=36, P100s=36, K80s=36; Jobs 4000-5000")
print("profiling_percentage=%f" % profiling_percentage)
average_jcts = compute_metric(v100s=36, p100s=36, k80s=36,
                              policies=all_policies,
                              profiling_percentage=profiling_percentage,
                              num_reference_models=16,
                              seeds=all_seeds)
plot_input_job_rate_vs_average_jct(average_jcts,
                                   output_filename='throughput_estimation/profiling_percentage=0.0625_v100=36_p100=36_k80=36.pdf')

In [ ]:
if reading_f is not None:
    reading_f.close()
if writing_f is not None:
    writing_f.close()